**Тема курсовой работы**: Система для сбора и генерации постов в социальных сетях

**Цель**: Построить модель машинного обучения, которая наиболее точно способна формировать текст, основываясь на начальных данных из обучающей выборки.

**Задачи**:

1. Провести предобработку и изучение данных.

2. Подготовить выборку.

3. Построить и обучить модель.

4. Cделать выводы о способностях модели генерировать текст.

**Ожидаемый результат**: \
После проведенной работы модель способна генерировать сложные предложения, исходя из существующей совокупности предложений

**Входные данные**:

Исходный dataset содержит выдержку текстов Льва Толстого на русском языке

**Практическая часть:**

Корпус, на котором тренируется модель- выборка из Льва Толстого (текстовый файл)

Из данного текста выделяем необходимую последовательность слов (функция get_tokens)

Получившийся генератор tokens будет выдавать «очищенную» последовательность слов и знаков препинания. Однако, нам интересны тройки токенов (слово или знак препинания, т.е. некие атомарные элементы текста). \
Для этого добавляем еще один генератор, на выходе которого будем иметь три подряд идущих токена (gen_trigrams).

gen_trigrams - cимволы '$' используются для маркировки начала предложения. В дальнейшем, это делает проще выбор первого слова генерируемой фразы. \
В целом, метод действует следующим образом: он возвращает три подряд идущих токена, на каждой итерации сдвигаясь на один токен: см. Пример 

Пример:

Вход:\
'Порок есть наказание без удовлетворения'\
Выход:\
0: $ $ порок\
1: $ порок есть\
2: порок есть наказание\
3: есть наказание без\
...

Далее, рассчитаем триграммную модель (train)

В первой части этого метода задаем генераторы. Далее, рассчитываем биграммы и триграммы (фактически, считаем количество одинаковых пар и троек слов в тексте). Далее, вычисляем вероятность слова в зависимости от двух предыдущих, помещая данное слово и его вероятность в словарь. \
Примечание: Это не самый оптимальный метод, так как идет значительный расход памяти. Но для небольших наборов данных этого вполне достаточно.

Теперь модель готова для генерации текста. Возвращаем предложение (generate_sentence)

Суть метода в последовательной выборке наиболее вероятных слов или знаков препинания до тех пор, пока не встретится признак начала следующей фразы (символа $). Первое слово выбирается как наиболее вероятное для начала предложения из набора.

Словарь model для каждой пары слов содержит список пар (слово, вероятность). Нам же необходимо выбрать из этого набора только одно слово. Если выбрать слово с максимальной вероятностью, тогда все сгенерированные фразы были бы похожи друг на друга. \
Более подходящий способ способ — выбирать слова с некой хаотичностью, которая бы зависела от вероятности слова (так как нам не нужно, чтобы наши фразы состояли из редких сочетаний). Это и делает метод unirand, который возвращает случайное слово с вероятностью, равной вероятности данного слова в зависимости от двух предыдущих.

In [3]:
with open(r'C:\Users\Админ\Downloads\tolstoy.txt', encoding='utf-8') as f:
    num_chars = len(re.sub('\s', '', f.read()))
    print (f' Файл содержит {num_chars} непустых символа')
    f.close()

 Файл содержит 3422485 непустых символа


In [10]:
# -*- coding: utf-8 -*-

import re
from random import uniform
from collections import defaultdict

r_alphabet = re.compile(u'[а-яА-Я0-9-]+|[.,:;?!]+')

def gen_lines(corpus):
    data = open(corpus, encoding='utf-8')
    for line in data:
        yield line.lower()

def gen_tokens(lines):
    for line in lines:
        for token in r_alphabet.findall(line):
            yield token

def gen_trigrams(tokens):
    t0, t1 = '$', '$'
    for t2 in tokens:
        yield t0, t1, t2
        if t2 in '.!?':
            yield t1, t2, '$'
            yield t2, '$','$'
            t0, t1 = '$', '$'
        else:
            t0, t1 = t1, t2

def train(corpus):
    lines = gen_lines(corpus)
    tokens = gen_tokens(lines)
    trigrams = gen_trigrams(tokens)

    bi, tri = defaultdict(lambda: 0.0), defaultdict(lambda: 0.0)

    for t0, t1, t2 in trigrams:
        bi[t0, t1] += 1
        tri[t0, t1, t2] += 1

    model = {}
    for (t0, t1, t2), freq in tri.items():
        if (t0, t1) in model:
            model[t0, t1].append((t2, freq/bi[t0, t1]))
        else:
            model[t0, t1] = [(t2, freq/bi[t0, t1])]
    return model

def generate_sentence(model):
    phrase = ''
    t0, t1 = '$', '$'
    while 1:
        t0, t1 = t1, unirand(model[t0, t1])
        if t1 == '$': break
        if t1 in ('.!?,;:') or t0 == '$':
            phrase += t1
        else:
            phrase += ' ' + t1
    return phrase.capitalize()

def unirand(seq):
    sum_, freq_ = 0, 0
    for item, freq in seq:
        sum_ += freq
    rnd = uniform(0, sum_)
    for token, freq in seq:
        freq_ += freq
        if rnd < freq_:
            return token

if __name__ == '__main__':
    model = train(r'C:\Users\Админ\Downloads\tolstoy.txt')
    print(generate_sentence(model))

Посылается генерал для осмотра позиции.


Дополнение:\
Триграммная модель выбрана, так как биграммы дают плохой результат, в то время как 4-граммы требуют существенно больше ресурсов.\
Расширить алгоритм для случая N-грамм возможно, но при увеличении числа N, текст будет все больше походить на исходный

**Вывод**: Наша модель успешно формирует предложения с сохранением правил русского языка и наличием смысла внутри